# Demo: Serenity Derivatives API - Option Pricing

Serenity builds in sophisticated option and rates analytics as part of its core offering, and these functions
are all exposed via the API. This notebook shows how you can use it to price European options on BTC, ETH and SOL.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2
#%run -i init_demo.py

In [ ]:
from serenity_sdk.renderers.connect_widget import ConnectWidget
connect_widget = ConnectWidget('athansor') # to connect automatically, insert the config id

In [ ]:
# create an alias to the api
api = connect_widget.get_api()

from datetime import datetime, timedelta
from uuid import UUID, uuid4
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from serenity_types.pricing.derivatives.rates.yield_curve import YieldCurveVersion
from serenity_types.pricing.derivatives.options.valuation import DiscountingMethod, OptionValuationRequest, OptionValuation

# helper modules
from serenity_sdk.renderers.derivatives.widget_tools import OptionChooser, YieldCurveVersionTimeChooser, VolatilitySurfaceVersionTimeChooser
from serenity_sdk.renderers.derivatives.table_plot import YieldCurveTablePlot, VolatilitySurfaceTablePlot, OptionValuationResultTablePlot
from serenity_sdk.renderers.derivatives.converters import convert_object_dict_to_df


# plot parameters
plt.rcParams['font.size'] = '16'

# Load samples of pre-defined option instruments

## Prepare a mapping from UUID to native symbol (linked asset native symbol)

In [ ]:
# Let's first remind that our asset ids are represented using uuids
# The mapping between asset id and native symbols are as below:
asset_summaries = api.refdata().get_asset_summaries()
asset_summaries = [{key: value for key, value in summary.items() if key != 'xrefSymbols'} for summary in asset_summaries]
asset_summaries = pd.json_normalize(asset_summaries)[['assetId', 'nativeSymbol']]
asset_summaries = asset_summaries[asset_summaries['nativeSymbol'].isin(['BTC','ETH'])]
asset_summaries

## Predefined option instruments from a csv file
For now, we read from a pre-saved csv file. In future, we plan to support to query predefined option instruments through an API. 
### Read & parse

In [ ]:
# Load sample options in the system & merge with the underlying asset id and symbol
sample_option_data_file = os.path.join('sample_data', 'list_options_20221202.csv')
sample_options = pd.read_csv(sample_option_data_file, parse_dates=['expiry_datetime'])\
    [['linked_asset_id', 'native_symbol', 'asset_id', 'option_type', 'expiry_datetime', 'strike_price','option_style']]\
    .sort_values(['linked_asset_id', 'expiry_datetime', 'strike_price'])
sample_options['expiry_datetime'] = sample_options['expiry_datetime'].dt.tz_localize(None)
sample_options = pd.merge(sample_options, asset_summaries, how='inner', left_on='linked_asset_id', right_on='assetId')
sample_options.drop('assetId', axis=1, inplace=True)
sample_options.rename(columns={'nativeSymbol':'linked_asset_native_symbol'}, inplace=True)

### Peak samples of predefined options

In [ ]:
sample_options.head(3)

## Select the option to use as a base line

In [ ]:
option_chooser = OptionChooser(sample_options)
print('Select an option to play with')
display(option_chooser.get_widget_to_display())

In [ ]:
# Show the details of the option selected
predefined_option_info = option_chooser.get_selected_option()
predefined_option_info

# Option Valuation

Notes: 
* API takes a list of option valuations and identified using 'option_valuation_id' UUID.
For users' convenience, we create a dictionary with human readable keys. 
* The 'values' of the diction are passed to the OptionValuationRequest. 
* Then, we will re-organise the valuation results in terms of the human reable keys

## Helper function for 'compute_option_valuations'

In [ ]:
# Function to run 'compute_option_valuations' and put the valuation results into a helper object 
# to format outputs
def run_compute_option_valuations(the_optvals, as_of_time=None):
    if as_of_time is None: 
        request = OptionValuationRequest(options=[v for v in the_optvals.values()])
    else:
        request = OptionValuationRequest(as_of_time=as_of_time, options=[v for v in the_optvals.values()])
    val_results = api.pricer().compute_option_valuations(request)

    # use a helper object for output formatting
    ovr_tp = OptionValuationResultTablePlot(val_results, the_optvals)
    return ovr_tp.results_table

## Using a pre-defined option and its replace
* Use the asset_id (uuid) of the pre-defined option to construct a option valuation object
* We constuct the identical option using the attributes of the pre-defined option

### Define demo option valuations

In [ ]:
# dictionary of option valuations
demo0_optvals = {}
demo0_qty = 10
# 1. use the predefined option
demo0_optvals['predefined'] = OptionValuation(
    option_valuation_id=str(uuid4()),
    qty = demo0_qty, 
    option_asset_id=UUID(predefined_option_info['asset_id']),
    contract_size=1
)

# 2. replicate the same option manually
demo1_optvals = {}
demo1_optvals['predefined'] = demo0_optvals['predefined']
demo1_optvals['predefined_replica'] = OptionValuation(
    option_valuation_id=str(uuid4()),
    qty = demo0_qty, 
    underlier_asset_id=predefined_option_info['linked_asset_id'],
    strike=predefined_option_info['strike_price'],
    expiry=predefined_option_info['expiry_datetime'],
    option_type=predefined_option_info['option_type'],
    option_style=predefined_option_info['option_style'],
    contract_size=1)


# show option vals
convert_object_dict_to_df(demo1_optvals)


### Run 'compute_option_valuations'

In [ ]:
res_table = run_compute_option_valuations(demo1_optvals)
print('Expected Behaviour: They should return the exactly the same results')
res_table

## With Market Data Overrides

Notes: At the time of producing this notebook. 
* Market data override implementations are WIP. 
* A version of spot bumps are there and there are still bugs to fix. 

This is to demonstrate how bumps would be working

### Define a set of option valuations with spot bumps and send them to API

In [ ]:
# market data bump
spot_bumps = np.array([-20.0, -10.0, -5.0, -2.5, -1.0, 0.0, +1.0, +2.5, +5.0, +10.0, +20.0])/100.0 + 1.0

# pick the base option to bump
base_optval = demo1_optvals['predefined_replica'].copy()

# dictionary of option valuations with market data bumped
spot_bumps_optvals = {}
spot_bumps_optvals['base'] = base_optval
for sb in spot_bumps:
    optval_this = base_optval.copy()
    optval_this.option_valuation_id=str(uuid4()) # need a unique id 
    optval_this.spot_price_bump=sb # set the bump
    spot_bumps_optvals[f'spot_bump_{sb}'] = optval_this

# show option valuations
convert_object_dict_to_df(spot_bumps_optvals)

### Run 'compute_option_valuations'

In [ ]:
res_table = run_compute_option_valuations(spot_bumps_optvals)
res_table.to_clipboard()

### Plot
* The first-order Taylor expansion using the delta from the base option should give a good approximation of the spot bump/revals
* At this point, we have some bug to fix. 

In [ ]:
qty = demo0_qty
delta_ccy = res_table['base']['delta_ccy']
plt.figure()
plt.plot(spot_bumps, 
    res_table[[c for c in res_table.columns if c!='base']].loc['pv']-res_table['base']['pv'],
    '.-', label='bump & pv change')
plt.plot(spot_bumps, 
    .2 * qty * res_table['base']['spot_price'] * res_table['base']['delta'] * (spot_bumps-1),
    '.-', label='1st-order approx (work-in-progress)')
plt.plot()
plt.grid()
plt.xlabel('bump'), plt.ylabel('pv change')
plt.legend()
plt.show()

MORE TO COME

# END